# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

#### load data

In [4]:
path = '/home/jovyan/work/original/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv'
data = spark.read.csv(path, inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


#### Prepare data

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [8]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [9]:
# change string to index using StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_idx")
indexed = indexer.fit(data).transform(data)

In [12]:
indexed.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_idx=16.0)

In [16]:
assembler = VectorAssembler(inputCols=['Cruise_line_idx', 'Age', 'Tonnage', 'passengers', 'length', 'cabins',
                                       'passenger_density'],
                            outputCol='features')
output = assembler.transform(indexed)

In [17]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_idx=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [18]:
final_data = output.select('features', 'crew')
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [19]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.961929824561407|
| stddev|3.5639264111452094|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [20]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean|7.3595454545454535|
| stddev| 3.342072156643453|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



#### Create and train model

In [21]:
lr = LinearRegression(labelCol="crew")

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.4133977113631495|
| -0.3545151821578596|
| -0.6512704077767104|
|-0.48220749849596967|
| -0.8017756156837885|
| -0.9270586290086449|
|-0.22932018282724975|
|  0.8006093769781746|
| -1.0681367134703663|
|  0.3030379614270817|
| -0.7066849464933771|
| -0.8191183956137067|
|  0.5615288915630003|
| 0.37062154212590936|
|  0.3821245476306956|
|-0.14777474530988854|
| 0.13458259794846228|
|  0.1863889204123752|
|-0.22254707885116165|
|-0.01753500868945...|
+--------------------+
only showing top 20 rows



In [25]:
test_results.rootMeanSquaredError

0.7146623001491669

In [28]:
from pyspark.sql.functions import corr

In [30]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

